# 문제 1) Tokenizer 생성하기

In [32]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False
  
    def preprocessing(self, sequences):
        result = []
        import nltk
        from nltk.tokenize import RegexpTokenizer
        tok=RegexpTokenizer('\w+')
        for sentence in sequences:
            sentence=sentence.lower()
            result.append(tok.tokenize(sentence))
        return result

#     def preprocessing(self, sequences):
#         result = []
#         tok=RegexpTokenizer('\w+')
#         for sentence in sequences:
#             sentence=sentence.lower()
#             result.append(tok.tokenize(sentence))
#             result.append(sentence.split(' '))
#         return result
  
    def fit(self, sequences):
        self.fit_checker = False
        result=self.preprocessing(sequences)
#         res = preprocessing(sentence)
        num=1
        for sentence in result:
            for word in sentence:
                if not word in self.word_dict.keys():
                    self.word_dict[word]=num
                    num+=1
        self.fit_checker = True
  
    def transform(self, sequences):
        result = []
        tokens = self.preprocessing(sequences)
        if self.fit_checker:
            for sentence in tokens:
                res = []
                for word in sentence:
                    if word in self.word_dict.keys():
                        res.append(self.word_dict[word])
                    else:
                        res.append(self.word_dict['oov'])
                result.append(res)

            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
      
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result

In [40]:
tokenizer=Tokenizer()
sent = ['I go to school.', 'I LIKE pizza!', 'I love Tteokpokki']
tokenizer.fit_transform(sent)
tokenizer.word_dict

{'oov': 0,
 'i': 1,
 'go': 2,
 'to': 3,
 'school': 4,
 'like': 5,
 'pizza': 6,
 'love': 7,
 'tteokpokki': 8}

# 문제 2) TfidfVectorizer 생성하기

In [43]:
import numpy as np
arr1 = [1,2,3,4,5,6]
arr2 = [[1,1,1,1,1,1],[2,2,2,2,2,2]]
np.multiply(arr1, arr2)

array([[ 1,  2,  3,  4,  5,  6],
       [ 2,  4,  6,  8, 10, 12]])

In [68]:
# corpus = sequences

class TfidfVectorizer:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
  
    
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        import math
        #tokenized=[[1,2,3,4], [1,5,6]]
        token_set=set()
        for sent in tokenized:
            for word in sent:
                token_set.add(word)
        tokens=list(token_set)
        tokens.sort()
        idf = []
        n=len(tokenized)
        for token in tokens:
            df = 0
            for sent in tokenized:
                if token in sent:
                    df += 1
            idf_val = math.log(n/(1+df))
            idf.append(idf_val)
            
        self.fit_checker = True
        return idf
        


    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)
            import numpy as np
            tf = []
            token_set=set()
            for sent in tokenized:
                for word in sent:
                    token_set.add(word)
            tokens=list(token_set)
            tokens.sort()
            for sentence in tokenized:
                temp = []
                for token in tokens:
                    tf_val = sum([word==token for word in sentence])
                    temp.append(tf_val)
                tf.append(temp)
            idf = self.fit(sequences)
            self.tfidf_matrix = np.multiply(tf,idf)
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")

  
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [67]:
tfidfvectorizer=TfidfVectorizer(tokenizer)
sent = ['I go to school.', 'I LIKE pizza!']
tfidfvectorizer.fit_transform(sent)

[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]


array([[-0.40546511,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.40546511,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])